In [1]:
import pandas as pd
import numpy as np
import requests
import json

In [2]:
#Keep this Secret
from config import aid, api_token, request_url

In [3]:
# Step 1: Get API Request
params = {
    "api_token": api_token,
    "aid": aid
}

headers = {
    "Content-Type": "application/json"
}

data = {
    "aid": aid
}

#API request
response = requests.post(request_url, headers=headers, json=data, params=params)

In [4]:
# Step 2: View JSON response
json_response = response.json()
print(json_response)

{'code': 0, 'ts': 1757337845, 'limit': 100, 'offset': 0, 'total': 9, 'count': 9, 'users': [{'first_name': '', 'last_name': '', 'personal_name': 'user_9@example.com', 'email': 'user_9@example.com', 'uid': 'PNIABp6mSpnhexq', 'image1': None, 'create_date': 1551080078, 'display_name': 'user_9@example.com', 'reset_password_email_sent': True, 'custom_fields': []}, {'first_name': '', 'last_name': '', 'personal_name': 'has@access.com', 'email': 'has@access.com', 'uid': 'PNIAHeDI8pjduak', 'image1': None, 'create_date': 1544194892, 'display_name': 'has@access.com', 'reset_password_email_sent': False, 'custom_fields': []}, {'first_name': '', 'last_name': '', 'personal_name': 'user_5@example.com', 'email': 'user_5@example.com', 'uid': 'PNIHNSuFZpnhey1', 'image1': None, 'create_date': 1551080089, 'display_name': 'user_5@example.com', 'reset_password_email_sent': True, 'custom_fields': []}, {'first_name': '', 'last_name': '', 'personal_name': '', 'email': 'newuser@piano.io', 'uid': 'PNIMzXlM3qu4y4y'

In [5]:
# Step 3: Extract users array and create DataFrame
if 'users' in json_response and isinstance(json_response['users'], list):
        df = pd.DataFrame(json_response['users'])

# Clean and strip columns
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].astype(str).str.strip()
        
#Save to CSV
df.to_csv('piano_users.csv', index=False)

In [6]:
# View piano_system df
piano_system = pd.read_csv('piano_users.csv')
piano_system

,first_name,last_name,personal_name,email,uid,image1,create_date,display_name,reset_password_email_sent,custom_fields
0,NaN,NaN,user_9@example.com,user_9@example.com,PNIABp6mSpnhexq,NaN,1551080078,user_9@example.com,True,[]
1,NaN,NaN,has@access.com,has@access.com,PNIAHeDI8pjduak,NaN,1544194892,has@access.com,False,[]
2,NaN,NaN,user_5@example.com,user_5@example.com,PNIHNSuFZpnhey1,NaN,1551080089,user_5@example.com,True,[]
3,NaN,NaN,NaN,newuser@piano.io,PNIMzXlM3qu4y4y,NaN,1622725523,newuser@piano.io,True,[]
4,Matt,Hinton,Matt Hinton,matt.hinton@piano.io,PNIP7FSyVpoky83,NaN,1552928259,matt hinton,True,[]
5,NaN,NaN,user_0@example.com,user_0@example.com,PNISGXVjCpnhexk,NaN,1551080072,user_0@example.com,True,[]
6,NaN,NaN,no@access.com,no@access.com,PNITyDyAOpjdubb,NaN,1544194919,no@access.com,False,[]
7,allegra,anka,allegra anka,allegra.anka@piano.io,PNIYDhiQRqayeba,NaN,1590507191,allegra anka,False,[]
8,J,Mathews,J Mathews,test-login@assessments.io,PNIwGolcnqhazws,NaN,1601172749,j mathews,True,[]


In [7]:
# View Merged CSV File
merged = pd.read_csv('merged_data.csv')
merged

,user_id,email,first_name,last_name
0,4dBdXURAz3,user_6@example.com,Jacques,Cuellar
1,fBYRtPtAlC,user_7@example.com,John,Smith
2,fOSjdLnNP3,user_9@example.com,Qiana,Walk
3,fQFLNRDae8,user_5@example.com,Vella,Lynam
4,fjM66woroy,user_0@example.com,Yadira,Irving
5,gXWj37JC5d,user_8@example.com,Shaun,Kreiger
6,oh4mHXh8zN,user_3@example.com,Julie,Mosser
7,oi6IhEzu9R,user_2@example.com,Leatrice,Oquinn
8,uxz2jFwr5I,user_1@example.com,Benito,Festa
9,zSbmdNiSHH,user_4@example.com,Taryn,Jaycox


In [8]:
# Step 4: Create final table by merging and updating the 'merged' table and the 'piano_system' table
final = merged.copy()

# Update user_id where email exists in piano_system
email_to_uid = piano_system.set_index('email')['uid'].to_dict()

for email, uid in email_to_uid.items():
    final.loc[final['email'] == email, 'user_id'] = uid

# Sort values and reset index
final = final.sort_values('email')
final = final.reset_index(drop=True)

# Create final CSV File
final.to_csv('final_table.csv', index=False)

In [19]:
#Final Output
final

,user_id,email,first_name,last_name
0,PNISGXVjCpnhexk,user_0@example.com,Yadira,Irving
1,uxz2jFwr5I,user_1@example.com,Benito,Festa
2,oi6IhEzu9R,user_2@example.com,Leatrice,Oquinn
3,oh4mHXh8zN,user_3@example.com,Julie,Mosser
4,zSbmdNiSHH,user_4@example.com,Taryn,Jaycox
5,PNIHNSuFZpnhey1,user_5@example.com,Vella,Lynam
6,4dBdXURAz3,user_6@example.com,Jacques,Cuellar
7,fBYRtPtAlC,user_7@example.com,John,Smith
8,gXWj37JC5d,user_8@example.com,Shaun,Kreiger
9,PNIABp6mSpnhexq,user_9@example.com,Qiana,Walk
